In [ ]:
# In your notebook cell
%pip install --upgrade sagemaker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.0 MB/s eta 0:00:00
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.242.0
    Uninstalling sagemaker-2.242.0:
      Successfully uninstalled sagemaker-2.242.0


In [ ]:
import sagemaker
from sagemaker.model import Model

# 1. Load train/test splits from S3
sess = sagemaker.Session()
bucket = sess.default_bucket()

# Register the model pointing to your S3 artifact
xgb_model = Model(
    model_data="s3://{bucket}/recs/output/model.tar.gz",
    image_uri=sagemaker.image_uris.retrieve("xgboost", sess.boto_region_name),
    role=role,
)

X_test = pd.read_parquet(f"s3://{bucket}/recs/test/X_test.parquet")

In [ ]:
# Deploy locally (no AWS charges)
predictor = xgb_model.deploy(
    initial_instance_count=1,
    instance_type="local",            # <-- local mode
    endpoint_name="demo-local-endpt",
    serializer=sagemaker.serializers.CSVSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),
)

# Test it
print(predictor.predict(X_test.iloc[:5].to_csv(index=False, header=False)))